### Hydraulic Design Calculations

The following calculations determine the required vessel dimensions based on hydraulic constraints:

In [ ]:
# Import calculation functions with handcalcs rendering
import sys
from pathlib import Path

# Add calculations module to path
calc_path = Path(project_root) / 'notebooks' / 'calculations'
if str(calc_path) not in sys.path:
    sys.path.insert(0, str(calc_path))

from vessel_sizing import (
    calculate_bed_volume,
    calculate_minimum_area,
    calculate_vessel_diameter,
    calculate_actual_area,
    calculate_linear_velocity,
    calculate_bed_depth,
    calculate_service_velocity,
    calculate_freeboard,
    calculate_vessel_height,
    calculate_aspect_ratio
)

#### Step 1: Bed Volume Calculation

In [ ]:
# Design parameters (from simulation inputs)
Q_m3_hr = design.get('flow_m3_hr', 100)      # Design flow rate in m³/hr
SV_hr = 16                                    # Service velocity (bed volumes/hour) - standard for RO pretreatment
LV_max_m_hr = 25                             # Maximum linear velocity in m/hr

# Calculate required bed volume using handcalcs
V_bed_m3, V_bed_L = calculate_bed_volume(Q_m3_hr, SV_hr)

print(f"\nDesign flow rate Q = {Q_m3_hr:.1f} m³/hr")
print(f"Required bed volume = {V_bed_m3:.2f} m³ = {V_bed_L:.0f} L")

#### Step 2: Minimum Cross-Sectional Area

In [ ]:
# Calculate minimum cross-sectional area for linear velocity constraint
A_min_m2 = calculate_minimum_area(Q_m3_hr, LV_max_m_hr)

print(f"\nMinimum cross-sectional area = {A_min_m2:.2f} m²")

#### Step 3: Vessel Diameter Selection

In [ ]:
# Calculate minimum diameter from area
D_min_m = calculate_vessel_diameter(A_min_m2)

# Round up to standard diameter (0.1m increments, max 2.4m for containerized systems)
import math
D_selected_m = min(2.4, math.ceil(D_min_m * 10) / 10)  # Round up to nearest 0.1m

# Calculate actual cross-sectional area
A_actual_m2 = calculate_actual_area(D_selected_m)

# Calculate actual linear velocity
LV_actual_m_hr = calculate_linear_velocity(Q_m3_hr, A_actual_m2)

print(f"\nMinimum diameter = {D_min_m:.2f} m")
print(f"Selected standard diameter = {D_selected_m:.2f} m")
print(f"Actual linear velocity = {LV_actual_m_hr:.1f} m/hr")

# Summary of vessel design
vessel_summary = [
    ['Vessel Diameter', format_with_units(D_selected_m, 'm', 2) if 'format_with_units' in locals() else f"{D_selected_m:.2f} m"],
    ['Bed Depth', format_with_units(h_bed_final_m, 'm', 2) if 'format_with_units' in locals() else f"{h_bed_final_m:.2f} m"],
    ['Bed Volume', format_with_units(V_bed_actual_m3, 'm³', 2) if 'format_with_units' in locals() else f"{V_bed_actual_m3:.2f} m³"],
    ['Freeboard Height', format_with_units(h_freeboard_m, 'm', 2) if 'format_with_units' in locals() else f"{h_freeboard_m:.2f} m"],
    ['Total Vessel Height', format_with_units(h_vessel_m, 'm', 2) if 'format_with_units' in locals() else f"{h_vessel_m:.2f} m"],
    ['Linear Velocity', format_with_units(LV_actual_m_hr, 'm/hr', 1) if 'format_with_units' in locals() else f"{LV_actual_m_hr:.1f} m/hr"],
    ['Service Velocity', format_with_units(SV_actual_hr, 'BV/hr', 1) if 'format_with_units' in locals() else f"{SV_actual_hr:.1f} BV/hr"],
    ['Aspect Ratio (L/D)', f"{aspect_ratio:.2f}"]
]

df_vessel = pd.DataFrame(vessel_summary, columns=['Parameter', 'Value'])
display(df_vessel.style.set_caption('SAC Vessel Design Summary'))

# Design constraints table
constraints = [
    ['Linear Velocity Limit', '25 m/hr'],
    ['Actual Linear Velocity', f"{LV_actual_m_hr:.1f} m/hr"],
    ['Minimum Bed Depth', '0.75 m'],
    ['Actual Bed Depth', f"{h_bed_final_m:.2f} m"],
    ['Container Diameter Limit', '2.4 m'],
    ['Actual Diameter', f"{D_selected_m:.2f} m"]
]

df_constraints = pd.DataFrame(constraints, columns=['Parameter', 'Value'])
display(df_constraints.style.set_caption('Design Constraints'))

In [ ]:
# Calculate bed depth from volume
h_bed_m = calculate_bed_depth(V_bed_m3, A_actual_m2)

# Minimum bed depth constraint
h_bed_min_m = 0.75  # Minimum for proper distribution

# Select actual bed depth (round to 0.05m)
h_bed_final_m = max(h_bed_min_m, math.ceil(h_bed_m * 20) / 20)  # Round up to nearest 0.05m

# Recalculate actual bed volume
V_bed_actual_m3 = A_actual_m2 * h_bed_final_m

# Calculate service flow rate in BV/hr
SV_actual_hr = calculate_service_velocity(Q_m3_hr, V_bed_actual_m3)

print(f"\nCalculated bed depth = {h_bed_m:.2f} m")
print(f"Selected bed depth = {h_bed_final_m:.2f} m")
print(f"Actual bed volume = {V_bed_actual_m3:.2f} m³")
print(f"Actual service velocity = {SV_actual_hr:.1f} BV/hr")

#### Step 5: Freeboard and Total Height

In [ ]:
# Freeboard for backwash expansion (100% for conservative design)
expansion_factor = 1.0  # 100% expansion factor
h_freeboard_m = calculate_freeboard(h_bed_final_m, expansion_factor)

# Total vessel height
h_distributor_m = 0.1  # Height for distributor/collector
h_vessel_m = calculate_vessel_height(h_bed_final_m, h_freeboard_m, h_distributor_m)

# Aspect ratio check
aspect_ratio = calculate_aspect_ratio(h_bed_final_m, D_selected_m)

print(f"\nFreeboard height = {h_freeboard_m:.2f} m")
print(f"Total vessel height = {h_vessel_m:.2f} m")
print(f"Aspect ratio (L/D) = {aspect_ratio:.2f}")

#### Vessel Design Summary

In [ ]:
# Summary of vessel design
vessel_summary = [
    ['Vessel Diameter', format_with_units(D_selected_m, 'm', 2) if 'format_with_units' in locals() else f"{D_selected_m:.2f} m"],
    ['Bed Depth', format_with_units(h_bed_final_m, 'm', 2) if 'format_with_units' in locals() else f"{h_bed_final_m:.2f} m"],
    ['Bed Volume', format_with_units(V_bed_actual_m3, 'm³', 2) if 'format_with_units' in locals() else f"{V_bed_actual_m3:.2f} m³"],
    ['Freeboard Height', format_with_units(h_freeboard_m, 'm', 2) if 'format_with_units' in locals() else f"{h_freeboard_m:.2f} m"],
    ['Total Vessel Height', format_with_units(h_vessel_m, 'm', 2) if 'format_with_units' in locals() else f"{h_vessel_m:.2f} m"],
    ['Linear Velocity', format_with_units(LV_actual_m_hr, 'm/hr', 1) if 'format_with_units' in locals() else f"{LV_actual_m_hr:.1f} m/hr"],
    ['Service Velocity', format_with_units(SV_actual_hr, 'BV/hr', 1) if 'format_with_units' in locals() else f"{SV_actual_hr:.1f} BV/hr"],
    ['Aspect Ratio (L/D)', f"{aspect_ratio:.2f}"]
]

df_vessel = pd.DataFrame(vessel_summary, columns=['Parameter', 'Value'])
display(df_vessel.style.set_caption('SAC Vessel Design Summary'))

# Design verification
print("\n✓ Design Verification:")
if LV_actual_m_hr <= 25:
    print(f"  Linear velocity {LV_actual_m_hr:.1f} ≤ 25 m/hr ✓")
else:
    print(f"  ⚠️ Linear velocity {LV_actual_m_hr:.1f} > 25 m/hr")

if h_bed_final_m >= 0.75:
    print(f"  Bed depth {h_bed_final_m:.2f} ≥ 0.75 m ✓")
else:
    print(f"  ⚠️ Bed depth {h_bed_final_m:.2f} < 0.75 m")

if D_selected_m <= 2.4:
    print(f"  Diameter {D_selected_m:.2f} ≤ 2.4 m (fits in container) ✓")
else:
    print(f"  ⚠️ Diameter {D_selected_m:.2f} > 2.4 m (exceeds container)")